### Least Recently Used algorithm; commonly used as cache replacement policy

Would need the size of the cache. A data structure to store the cache and operations to rectrieve that data.First notion would be to somehow use a hashmap. Since it allows O(1) access times for insertions and retrieval. Needs more testing and code optimization and cleanup. Based upon the few tests run, looks good!

### Notes:
Cache has a specific user defined size. Get and Set operations implemented

### Approach:
###### Get:
1. Key is absent; return -1
2. Key is present; update LRU policy
        updateLRUPolicyonGet:
        if Required key is already at the top; do nothing
        else move the key to the top and shift all other keys down by 1 rank.

###### Set:
1. Key is already present; consider this as a get operation & update LRU policy as above
2. Key is absent, hence needs to be added; updateLRU policy then
        updateLRUPolicyonSet:
        if cache is not full; no need to remove any previous lru key. shift all keys down by 1 rank and add the new key to the top. 
        if cache is full; get the lru key; shift all keys down by 1; add the new key; remove the lru key
        
#### Data Structures Used:
##### HashMaps 

1. ranks_vs_keys: Used to maintain the main LRU policy to determine the order of recently accessed keys
2. keys_vs_ranks: Used to maintain a mapping of the ranks of the keys in terms of recently used
3. cache: main datastore for key:value pairs

### Analysis
###### Get Operation:
1. Worst Case: O(n)  ||  When the key requested is the least recently used. Need to shift all above keys down by 1 rank
2. Best Case: O(1) || Key request is the most recently used

###### Set Operation:
1. Worst Case & Average Case: O(n) || New element and cache is full, requires shifting of all elements down
2. Best Case: O(1) || Cache is empty

### Bugs
1. Now handling same keys with different values

In [16]:
import json

In [147]:
class LRUCache:

    def __init__(self, capacity):
        self.size = capacity
        self.used_capacity = 0
        # store hashedkeys
        self.ranks_vs_keys = {}
        self.keys_vs_ranks = {}
        self.cache = {}
        # store raw keys
        self.keys_count = {}
        self.keys_vs_hashes = {}
        
    def get(self, key):
        if key not in self.keys_count:
            return -1
        else:
            hashed_keys = self.generateHashedKeys(key)
            mru_key = self.getMRUHash(hashed_keys)
            self.updateLRUOnGet(mru_key)
            val = self.cache[mru_key]
        print(self)
        return val   

    def put(self, key, value):
        if key in self.keys_count:
            hashed_keys = self.generateHashedKeys(key)
            if self.sameKeyPair(hashed_keys, value):
                _ = self.updateLRUOnGet(key)
            else:
                self.keys_count[key] += 1
                key_hash = self.gethash(key, self.keys_count[key])
                self.keys_vs_hashes[key].add(key_hash)
                self.updateLRUOnSet(key_hash, value)
        else:
            self.keys_count[key] = 1
            key_hash = self.gethash(key, self.keys_count[key])
            self.keys_vs_hashes[key] = set([key_hash])
            self.updateLRUOnSet(key_hash, value)
        print(self)
        
    def sameKeyPair(self, hashed_keys, value):
        for each_hash in hashed_keys:
            if value == self.cache[each_hash]:
                return True
        return False

    def generateHashedKeys(self, key):
        keys_count = self.keys_count[key]
        result = set()
        for i in range(1, keys_count+1):
            result.add(self.gethash(key, i))
        return result

    def gethash(self, val1, val2):
        res = str(hash(str(val1)) + hash(str(val2)))
        return res

    def getMRUHash(self, hashed_keys):
        max_rank = -1
        for each_hash in hashed_keys:
            if self.keys_vs_ranks[each_hash] > max_rank:
                max_rank = self.keys_vs_ranks[each_hash]
        return self.ranks_vs_keys[max_rank]
        
    def updateLRUOnGet(self, key):
        if self.ranks_vs_keys[self.size] == key:
            return
        else:
            # update ranks of all keys above the rank of the key in the get request.
            curr_key_rank = self.keys_vs_ranks[key]
            new_key = key
            for rank in range(self.size, curr_key_rank - 1, -1):
                curr_key = self.ranks_vs_keys[rank]
                self.ranks_vs_keys[rank] = new_key
                self.keys_vs_ranks[new_key] = rank
                new_key = curr_key
    
    def updateLRUOnSet(self, key, val):
        if self.used_capacity < self.size:
            if self.used_capacity != 0:
                for rank in range(self.size - self.used_capacity, self.size):
                    next_key = self.ranks_vs_keys[rank + 1]
                    self.ranks_vs_keys[rank] = next_key
                    self.keys_vs_ranks[next_key] = rank
            self.ranks_vs_keys[self.size] = key
            self.keys_vs_ranks[key] = self.size
            self.used_capacity += 1
        else:
            least_recently_used_key = self.ranks_vs_keys[1]
            new_key = key
            for rank in range(self.size, 0, -1):
                curr_key = self.ranks_vs_keys[rank]
                self.ranks_vs_keys[rank] = new_key
                self.keys_vs_ranks[new_key] = rank
                new_key = curr_key
            del self.cache[least_recently_used_key]
            del self.keys_vs_ranks[least_recently_used_key]
            self.updateKeyCount(least_recently_used_key)
        self.cache[key] = val
        
    def updateKeyCount(self, lru_key):
        for key, val in self.keys_vs_hashes.iteritems():
            if lru_key in val:
                val.remove(lru_key)
                self.keys_count[key] -= 1
                break
        if self.keys_count[key] == 0:
            del self.keys_count[key]
            del self.keys_vs_hashes[key]

    def __repr__(self):
        return "KEYS_COUNT: %s\nCACHE: %s\nRANKS vs KEYS: %s\nKEYS vs RANKS: %s\nKEYS vs HASH: %s" % (json.dumps(self.keys_count),
                                                                                    json.dumps(self.cache),
                                                                                    json.dumps(self.ranks_vs_keys),
                                                                                    json.dumps(self.keys_vs_ranks),
                                                                                    self.keys_vs_hashes)

In [148]:
l = LRUCache(2)

In [149]:
l.put(2, 1)

KEYS_COUNT: {"2": 1}
CACHE: {"12672038115": 1}
RANKS vs KEYS: {"2": "12672038115"}
KEYS vs RANKS: {"12672038115": 2}
KEYS vs HASH: {2: set(['12672038115'])}


In [150]:
l.put(2,2)

KEYS_COUNT: {"2": 2}
CACHE: {"12800038502": 2, "12672038115": 1}
RANKS vs KEYS: {"1": "12672038115", "2": "12800038502"}
KEYS vs RANKS: {"12800038502": 2, "12672038115": 1}
KEYS vs HASH: {2: set(['12800038502', '12672038115'])}


In [151]:
l.get(2)

KEYS_COUNT: {"2": 2}
CACHE: {"12800038502": 2, "12672038115": 1}
RANKS vs KEYS: {"1": "12672038115", "2": "12800038502"}
KEYS vs RANKS: {"12800038502": 2, "12672038115": 1}
KEYS vs HASH: {2: set(['12800038502', '12672038115'])}


2

In [152]:
l.put(7,14)

KEYS_COUNT: {"2": 1, "7": 1}
CACHE: {"13312040038": 14, "12800038502": 2}
RANKS vs KEYS: {"1": "12800038502", "2": "13312040038"}
KEYS vs RANKS: {"13312040038": 2, "12800038502": 1}
KEYS vs HASH: {2: set(['12800038502']), 7: set(['13312040038'])}


In [153]:
l.get(7)

KEYS_COUNT: {"2": 1, "7": 1}
CACHE: {"13312040038": 14, "12800038502": 2}
RANKS vs KEYS: {"1": "12800038502", "2": "13312040038"}
KEYS vs RANKS: {"13312040038": 2, "12800038502": 1}
KEYS vs HASH: {2: set(['12800038502']), 7: set(['13312040038'])}


14

In [154]:
l.get(36)

-1

In [155]:
l.get(33)

-1

In [156]:
l.get(10)

-1

In [157]:
l.put(5,17)

KEYS_COUNT: {"5": 1, "7": 1}
CACHE: {"13312040038": 14, "13056039268": 17}
RANKS vs KEYS: {"1": "13312040038", "2": "13056039268"}
KEYS vs RANKS: {"13312040038": 1, "13056039268": 2}
KEYS vs HASH: {5: set(['13056039268']), 7: set(['13312040038'])}
